You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document

Title: The Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its significance in today's world
- Mention of the rapid advancements and ongoing developments in the field

Key Points:
1. Latest Trends in Artificial Intelligence
- Emphasis on the rise of machine learning and deep learning technologies
- Discussion on the increasing integration of AI in various industries such as healthcare, finance, and transportati

I now can give a great answer.

Final Answer: 

# The Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a fundamental component of our modern society, revolutionizing numerous industries and reshaping the way we interact with technology. With its rapid advancements and ongoing developments, AI stands at the forefront of innovation, catalyzing progress and efficiency across a wide array of applications.

## Latest Trends in Artificial Intelligence
A prominent trend in AI is the ascension of machine learning and deep learning technologies. These breakthroughs have empowered computers to learn from data and make intelligent decisions, resulting in significant advancements in areas like image recognition, natural language processing, and predictive analytics. The integration of AI in sectors such as healthcare, finance, and transportation has brought about substantial enhancements in efficiency, accuracy, and decision-making p

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a fundamental component of our modern society, revolutionizing numerous industries and reshaping the way we interact with technology. With its rapid advancements and ongoing developments, AI stands at the forefront of innovation, catalyzing progress and efficiency across a wide array of applications.

## Latest Trends in Artificial Intelligence
A prominent trend in AI is the ascension of machine learning and deep learning technologies. These breakthroughs have empowered computers to learn from data and make intelligent decisions, resulting in significant advancements in areas like image recognition, natural language processing, and predictive analytics. The integration of AI in sectors such as healthcare, finance, and transportation has brought about substantial enhancements in efficiency, accuracy, and decision-making processes. Nevertheless, this integration also raises concerns about job automation and the reconfiguration of the workforce, as AI systems progressively assume responsibilities traditionally carried out by humans.

## Key Players in the AI Industry
Pioneering companies like Google, Amazon, Microsoft, and IBM are spearheading AI research and development, propelling innovation and expanding the boundaries of what is achievable with artificial intelligence. These industry leaders have made notable contributions to the field, crafting cutting-edge technologies and solutions that hold the potential to revolutionize industries and enhance the quality of our lives and work. Furthermore, partnerships and collaborations within the industry have played a pivotal role in advancing AI capabilities, enabling companies to harness one another's strengths and resources to drive innovation and attain common objectives.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI technology have encompassed progress in areas such as autonomous vehicles, healthcare diagnostics, and personalized recommendations. Significant AI conferences and events have provided platforms for researchers, industry experts, and enthusiasts to exchange knowledge, ideas, and explore the latest trends and developments in artificial intelligence. Looking ahead, the future of AI brims with promise, yet it also poses challenges such as ethical considerations, regulatory frameworks, and societal impact, which will necessitate attention as the technology continues to evolve.

## Conclusion
As AI continues to mold the future of technology and society, keeping abreast of the latest trends and key players in the industry is imperative for professionals, business leaders, students, and enthusiasts alike. By delving into the dynamic landscape of artificial intelligence and grasping its implications for diverse industries and applications, readers can glean valuable insights and pinpoint opportunities for career advancement and innovation in this swiftly evolving field. I urge you to delve deeper into the realm of AI, explore its possibilities, and contribute to the exhilarating odyssey of artificial intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [21]:
topic = "Time Travel"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Time Travel.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: Exploring the Possibilities of Time Travel

Introduction:
- Brief explanation of the concept of time travel
- Mention of popular culture references to time travel
- Statement of the purpose of the article

Key Points:
1. Latest Trends in Time Travel:
- Discussion of current scientific research and theories on time travel
- Exploration of advancements in technology that may make time travel possible in the future
- Examples of recent breakthroughs in the field of quantum physics related to time trave

I now can give a great answer.

Final Answer:

# Exploring the Possibilities of Time Travel

## Introduction

The concept of time travel has long been a source of fascination, captivating both scientific minds and popular culture enthusiasts alike. From the beloved "Back to the Future" films to the thought-provoking theories proposed by leading physicists, the idea of journeying through time has sparked curiosity worldwide. In this article, we will delve into the latest developments, influential figures, and significant news within the realm of time travel, shedding light on the advancements and debates surrounding this intriguing topic.

## Latest Trends in Time Travel

In recent years, scientific exploration of time travel has gained momentum, with experts delving into cutting-edge theories and technologies that hold the promise of making time travel a tangible reality. Progress in quantum physics, particularly in the study of wormholes and black holes, has opened up new avenues for 

In [22]:
Markdown(result)

# Exploring the Possibilities of Time Travel

## Introduction

The concept of time travel has long been a source of fascination, captivating both scientific minds and popular culture enthusiasts alike. From the beloved "Back to the Future" films to the thought-provoking theories proposed by leading physicists, the idea of journeying through time has sparked curiosity worldwide. In this article, we will delve into the latest developments, influential figures, and significant news within the realm of time travel, shedding light on the advancements and debates surrounding this intriguing topic.

## Latest Trends in Time Travel

In recent years, scientific exploration of time travel has gained momentum, with experts delving into cutting-edge theories and technologies that hold the promise of making time travel a tangible reality. Progress in quantum physics, particularly in the study of wormholes and black holes, has opened up new avenues for manipulating time. Breakthroughs in quantum computing and teleportation have further expanded the horizons of what was once considered beyond the realm of possibility. As we continue to unravel the mysteries of the cosmos, the feasibility of time travel becomes increasingly conceivable.

## Key Players in Time Travel

Distinguished scientists and researchers within the field of time travel play a pivotal role in shaping our comprehension of this intricate phenomenon. Visionaries such as Dr. Michio Kaku and Dr. Ronald Mallett have devoted their expertise to unraveling the enigmas of time travel, offering valuable insights and perspectives on the subject. Institutions like the Time Travel Institute and the Perimeter Institute for Theoretical Physics have served as catalysts for collaboration and innovation in the field. Through engaging with these experts via interviews, we can gain a deeper insight into the current landscape of time travel research and the obstacles that lie ahead.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).